In [78]:
%matplotlib inline
import tensorflow as tf
import numpy as py
import pandas as pd

In [79]:
with open("C:/Users/Bin/Documents/Datasets/KDD99/columns.txt") as col_file:
    line = col_file.readline()

columns = line.split('.')
col_names = []
col_types = []
for col in columns:
    col_names.append(col.split(': ')[0].strip())
    col_types.append(col.split(': ')[1])
col_names.append("label")

df = pd.read_csv("C:/Users/Bin/Documents/Datasets/KDD99/kddcup.data_10_percent_corrected",names=col_names)

data = df.iloc[:5000,np.array(pd.Series(col_types)=="continuous")].as_matrix()
label = df.iloc[:,-1]

In [80]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
dataset = scaler.transform(data)
del data

In [81]:
dataset.shape

(5000, 34)

In [29]:
timesteps = 30
latent_dim = 10
n_epoch = 100  #num_steps
n_batch = 100

In [74]:
def batch_producer(raw_data, batch_size, num_steps):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

    data_len = tf.size(raw_data.shape[0])
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0: batch_size * batch_len,:],
                      [batch_size, batch_len,raw_data.shape[1]])

    epoch_size = (batch_len - 1) // num_steps

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = data[:, i * num_steps:(i + 1) * num_steps,:]
    x.set_shape([batch_size, num_steps,raw_data.shape[1]])
    #y = data[:, i * num_steps + 1: (i + 1) * num_steps + 1]
    #y.set_shape([batch_size, num_steps])
    return x

In [77]:
class Input(object):
    def __init__(self, batch_size, num_steps, data):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
        self.input_data, self.targets = batch_producer(data, batch_size, num_steps)

In [ ]:
class Model(object):
    def __init__(self, input, is_training, hidden_size, vocab_size, num_layers,
                 dropout=0.5, init_scale=0.05):
        self.is_training = is_training
        self.input_obj = input
        self.batch_size = input.batch_size
        self.num_steps = input.num_steps
    inputs =     
    if is_training and dropout < 1:
    inputs = tf.nn.dropout(inputs, dropout)
    
    # set up the state storage / extraction
    self.init_state = tf.placeholder(tf.float32, [num_layers, 2, self.batch_size, self.hidden_size])
    
    state_per_layer_list = tf.unstack(self.init_state, axis=0)
    rnn_tuple_state = tuple(
            [tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
             for idx in range(num_layers)]
        )
    
    # create an LSTM cell to be unrolled
    cell = tf.contrib.rnn.LSTMCell(hidden_size, forget_bias=1.0)
    # add a dropout wrapper if training
    if is_training and dropout < 1:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=dropout)
    if num_layers > 1:
        cell = tf.contrib.rnn.MultiRNNCell([cell for _ in range(num_layers)], state_is_tuple=True) #expect the state variables in the form of a LSTMStateTuple 
   
    output, self.state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, initial_state=rnn_tuple_state)
    # output size:  (batch_size, num_steps, hidden_size)

In [ ]:
def assign_lr(self, session, lr_value):
    session.run(self.lr_update, feed_dict={self.new_lr: lr_value})
    
def train(train_data, vocabulary, num_layers, num_epochs, batch_size, model_save_name,
          learning_rate=1.0, max_lr_epoch=10, lr_decay=0.93):
    # setup data and models
    training_input = Input(batch_size=batch_size, num_steps=35, data=train_data)
    m = Model(training_input, is_training=True, hidden_size=650, vocab_size=vocabulary,
              num_layers=num_layers)
    init_op = tf.global_variables_initializer()
    
    orig_decay = lr_decay
    with tf.Session() as sess:
        # start threads
        sess.run([init_op])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        saver = tf.train.Saver()

    for epoch in range(num_epochs):
        new_lr_decay = orig_decay ** max(epoch + 1 - max_lr_epoch, 0.0)
        m.assign_lr(sess, learning_rate * new_lr_decay)
        current_state = np.zeros((num_layers, 2, batch_size, m.hidden_size))
        for step in range(training_input.epoch_size):
            if step % 50 != 0:
                cost, _, current_state = sess.run([m.cost, m.train_op, m.state],
                                                                 feed_dict={m.init_state: current_state})
            else:
                cost, _, current_state, acc = sess.run([m.cost, m.train_op, m.state, m.accuracy],
                                                          feed_dict={m.init_state: current_state})
                print("Epoch {}, Step {}, cost: {:.3f}, accuracy: {:.3f}".format(epoch, step, cost, acc))
        # save a model checkpoint
        saver.save(sess, data_path + '\\' + model_save_name, global_step=epoch)
    # do a final save
    saver.save(sess, data_path + '\\' + model_save_name + '-final')
    # close threads
    coord.request_stop()
    coord.join(threads)

In [82]:
import tensorflow as tf

import numpy as np

import collections

import os

import argparse

import datetime as dt



"""To run this code, you'll need to first download and extract the text dataset

    from here: http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz. Change the

    data_path variable below to your local exraction path"""



data_path = "C:/Users/Bin/Desktop/exp/simple-examples/data"



parser = argparse.ArgumentParser()

parser.add_argument('run_opt', type=int, default=1, help='An integer: 1 to train, 2 to test')

parser.add_argument('--data_path', type=str, default=data_path, help='The full path of the training data')

args = parser.parse_args()



def read_words(filename):

    with tf.gfile.GFile(filename, "r") as f:

        return f.read().decode("utf-8").replace("\n", "<eos>").split()





def build_vocab(filename):

    data = read_words(filename)



    counter = collections.Counter(data)

    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))



    words, _ = list(zip(*count_pairs))

    word_to_id = dict(zip(words, range(len(words))))



    return word_to_id





def file_to_word_ids(filename, word_to_id):

    data = read_words(filename)

    return [word_to_id[word] for word in data if word in word_to_id]





def load_data():

    # get the data paths

    train_path = os.path.join(data_path, "ptb.train.txt")

    valid_path = os.path.join(data_path, "ptb.valid.txt")

    test_path = os.path.join(data_path, "ptb.test.txt")



    # build the complete vocabulary, then convert text data to list of integers

    word_to_id = build_vocab(train_path)

    train_data = file_to_word_ids(train_path, word_to_id)

    valid_data = file_to_word_ids(valid_path, word_to_id)

    test_data = file_to_word_ids(test_path, word_to_id)

    vocabulary = len(word_to_id)

    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))



    print(train_data[:5])

    print(word_to_id)

    print(vocabulary)

    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))

    return train_data, valid_data, test_data, vocabulary, reversed_dictionary





def batch_producer(raw_data, batch_size, num_steps):

    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)



    data_len = tf.size(raw_data)

    batch_len = data_len // batch_size

    data = tf.reshape(raw_data[0: batch_size * batch_len],

                      [batch_size, batch_len])



    epoch_size = (batch_len - 1) // num_steps



    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()

    x = data[:, i * num_steps:(i + 1) * num_steps]

    x.set_shape([batch_size, num_steps])

    y = data[:, i * num_steps + 1: (i + 1) * num_steps + 1]

    y.set_shape([batch_size, num_steps])

    return x, y





class Input(object):

    def __init__(self, batch_size, num_steps, data):

        self.batch_size = batch_size

        self.num_steps = num_steps

        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps

        self.input_data, self.targets = batch_producer(data, batch_size, num_steps)





# create the main model

class Model(object):

    def __init__(self, input, is_training, hidden_size, vocab_size, num_layers,

                 dropout=0.5, init_scale=0.05):

        self.is_training = is_training

        self.input_obj = input

        self.batch_size = input.batch_size

        self.num_steps = input.num_steps

        self.hidden_size = hidden_size



        # create the word embeddings

        with tf.device("/cpu:0"):

            embedding = tf.Variable(tf.random_uniform([vocab_size, self.hidden_size], -init_scale, init_scale))

            inputs = tf.nn.embedding_lookup(embedding, self.input_obj.input_data)



        if is_training and dropout < 1:

            inputs = tf.nn.dropout(inputs, dropout)



        # set up the state storage / extraction

        self.init_state = tf.placeholder(tf.float32, [num_layers, 2, self.batch_size, self.hidden_size])



        state_per_layer_list = tf.unstack(self.init_state, axis=0)

        rnn_tuple_state = tuple(

            [tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])

             for idx in range(num_layers)]

        )



        # create an LSTM cell to be unrolled

        cell = tf.contrib.rnn.LSTMCell(hidden_size, forget_bias=1.0)

        # add a dropout wrapper if training

        if is_training and dropout < 1:

            cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=dropout)

        if num_layers > 1:

            cell = tf.contrib.rnn.MultiRNNCell([cell for _ in range(num_layers)], state_is_tuple=True)



        output, self.state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, initial_state=rnn_tuple_state)

        # reshape to (batch_size * num_steps, hidden_size)

        output = tf.reshape(output, [-1, hidden_size])



        softmax_w = tf.Variable(tf.random_uniform([hidden_size, vocab_size], -init_scale, init_scale))

        softmax_b = tf.Variable(tf.random_uniform([vocab_size], -init_scale, init_scale))

        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

        # Reshape logits to be a 3-D tensor for sequence loss

        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])



        # Use the contrib sequence loss and average over the batches

        loss = tf.contrib.seq2seq.sequence_loss(

            logits,

            self.input_obj.targets,

            tf.ones([self.batch_size, self.num_steps], dtype=tf.float32),

            average_across_timesteps=False,

            average_across_batch=True)



        # Update the cost

        self.cost = tf.reduce_sum(loss)



        # get the prediction accuracy

        self.softmax_out = tf.nn.softmax(tf.reshape(logits, [-1, vocab_size]))

        self.predict = tf.cast(tf.argmax(self.softmax_out, axis=1), tf.int32)

        correct_prediction = tf.equal(self.predict, tf.reshape(self.input_obj.targets, [-1]))

        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



        if not is_training:

           return

        self.learning_rate = tf.Variable(0.0, trainable=False)



        tvars = tf.trainable_variables()

        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), 5)

        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)

        # optimizer = tf.train.AdamOptimizer(self.learning_rate)

        self.train_op = optimizer.apply_gradients(

            zip(grads, tvars),

            global_step=tf.contrib.framework.get_or_create_global_step())

        # self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.cost)



        self.new_lr = tf.placeholder(tf.float32, shape=[])

        self.lr_update = tf.assign(self.learning_rate, self.new_lr)



    def assign_lr(self, session, lr_value):

        session.run(self.lr_update, feed_dict={self.new_lr: lr_value})





def train(train_data, vocabulary, num_layers, num_epochs, batch_size, model_save_name,

          learning_rate=1.0, max_lr_epoch=10, lr_decay=0.93, print_iter=50):

    # setup data and models

    training_input = Input(batch_size=batch_size, num_steps=35, data=train_data)

    m = Model(training_input, is_training=True, hidden_size=650, vocab_size=vocabulary,

              num_layers=num_layers)

    init_op = tf.global_variables_initializer()

    orig_decay = lr_decay

    with tf.Session() as sess:

        # start threads

        sess.run([init_op])

        coord = tf.train.Coordinator()

        threads = tf.train.start_queue_runners(coord=coord)

        saver = tf.train.Saver()

        for epoch in range(num_epochs):

            new_lr_decay = orig_decay ** max(epoch + 1 - max_lr_epoch, 0.0)

            m.assign_lr(sess, learning_rate * new_lr_decay)

            # m.assign_lr(sess, learning_rate)

            # print(m.learning_rate.eval(), new_lr_decay)

            current_state = np.zeros((num_layers, 2, batch_size, m.hidden_size))

            curr_time = dt.datetime.now()

            for step in range(training_input.epoch_size):

                # cost, _ = sess.run([m.cost, m.optimizer])

                if step % print_iter != 0:

                    cost, _, current_state = sess.run([m.cost, m.train_op, m.state],

                                                      feed_dict={m.init_state: current_state})

                else:

                    seconds = (float((dt.datetime.now() - curr_time).seconds) / print_iter)

                    curr_time = dt.datetime.now()

                    cost, _, current_state, acc = sess.run([m.cost, m.train_op, m.state, m.accuracy],

                                                           feed_dict={m.init_state: current_state})

                    print("Epoch {}, Step {}, cost: {:.3f}, accuracy: {:.3f}, Seconds per step: {:.3f}".format(epoch,

                            step, cost, acc, seconds))



            # save a model checkpoint

            saver.save(sess, data_path + '\\' + model_save_name, global_step=epoch)

        # do a final save

        saver.save(sess, data_path + '\\' + model_save_name + '-final')

        # close threads

        coord.request_stop()

        coord.join(threads)





def test(model_path, test_data, reversed_dictionary):

    test_input = Input(batch_size=20, num_steps=35, data=test_data)

    m = Model(test_input, is_training=False, hidden_size=650, vocab_size=vocabulary,

              num_layers=2)

    saver = tf.train.Saver()

    with tf.Session() as sess:

        # start threads

        coord = tf.train.Coordinator()

        threads = tf.train.start_queue_runners(coord=coord)

        current_state = np.zeros((2, 2, m.batch_size, m.hidden_size))

        # restore the trained model

        saver.restore(sess, model_path)

        # get an average accuracy over num_acc_batches

        num_acc_batches = 30

        check_batch_idx = 25

        acc_check_thresh = 5

        accuracy = 0

        for batch in range(num_acc_batches):

            if batch == check_batch_idx:

                true_vals, pred, current_state, acc = sess.run([m.input_obj.targets, m.predict, m.state, m.accuracy],

                                                               feed_dict={m.init_state: current_state})

                pred_string = [reversed_dictionary[x] for x in pred[:m.num_steps]]

                true_vals_string = [reversed_dictionary[x] for x in true_vals[0]]

                print("True values (1st line) vs predicted values (2nd line):")

                print(" ".join(true_vals_string))

                print(" ".join(pred_string))

            else:

                acc, current_state = sess.run([m.accuracy, m.state], feed_dict={m.init_state: current_state})

            if batch >= acc_check_thresh:

                accuracy += acc

        print("Average accuracy: {:.3f}".format(accuracy / (num_acc_batches-acc_check_thresh)))

        # close threads

        coord.request_stop()

        coord.join(threads)





if args.data_path:

    data_path = args.data_path

train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()

if args.run_opt == 1:

    train(train_data, vocabulary, num_layers=2, num_epochs=60, batch_size=20,

          model_save_name='two-layer-lstm-medium-config-60-epoch-0p93-lr-decay-10-max-lr')

else:

    trained_model = args.data_path + "\\two-layer-lstm-medium-config-60-epoch-0p93-lr-decay-10-max-lr-38"

    test(trained_model, test_data, reversed_dictionary)

usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH] run_opt
ipykernel_launcher.py: error: argument run_opt: invalid int value: 'C:\\Users\\Bin\\AppData\\Roaming\\jupyter\\runtime\\kernel-78a79ad2-fa2d-482c-a768-b46d26cbc945.json'


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
